Two-Armed Bandit

In [1]:
import tensorflow as tf
import numpy as np

In [11]:
# List out our bandist. Current bandit 5 (index#3) is set to most often provide a reward
bandits = [0.2, 0, -0.2, -5]
num_bandits = len(bandits)


def pullBandit(bandit):
    # Get a random number.
    result = np.random.randn(1)
    if result > bandit:
        # Return a positive reward.
        return 1
    else:
        # Return a negative reward
        return -1


In [4]:
tf.reset_default_graph()

# These two lines established the feed-forward part of the network. This does the actual choosing.
weights = tf.Variable(tf.ones([num_bandits]))
chosen_action = tf.argmax(weights, 0)

# The next size lines establish the training procedure. We feed the reward and chosen action into the network to compute the loss, and use it to update 
# the network
reward_holder = tf.placeholder(shape=[1], dtype=tf.float32)
action_holder = tf.placeholder(shape=[1], dtype=tf.int32)
responsible_weight = tf.slice(weights, action_holder, [1])
loss = -(tf.log(responsible_weight)*reward_holder)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
update = optimizer.minimize(loss)

In [12]:
total_episodes = 1000 # Set total number of episodes to train agent on
total_reward = np.zeros(num_bandits) # SEt scoreboard for bandits to 0
e = 0.1 #Set the chance of tkaing a random action

init = tf.initialize_all_variables()

# Launch the tensorflow graph
with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        # Choose either a random action or one from our network.
        if np.random.rand(1) < e:
            action = np.random.randint(num_bandits)
        else:
            action = sess.run(chosen_action)
        reward = pullBandit(bandits[action]) # Get our reward from picking one of the bandits.
        # Update the network
        _, resp, ww = sess.run([update, responsible_weight, weights], feed_dict={reward_holder: [reward], 
                                                                                 action_holder: [action]})
        # Update our running tally of scores
        total_reward[action] += reward
        if i % 50 == 0:
            print("Running reward for the " + str(num_bandits) + " bandits: " + str(total_reward))
        i +=1
print("The agent thinks bandit " + str(np.argmax(ww)+1) + " is the most promising....")
if np.argmax(ww) == np.argmax(-np.array(bandits)):
    print("...and it was right!")
else:
    print("...and it was wrong!")

Running reward for the 4 bandits: [-1.  0.  0.  0.]
Running reward for the 4 bandits: [-2. 44. -2.  1.]
Running reward for the 4 bandits: [-2. 90.  0.  3.]
Running reward for the 4 bandits: [ -2. 136.   2.   5.]
Running reward for the 4 bandits: [ -2. 184.   2.   5.]


Running reward for the 4 bandits: [ -1. 229.   4.   7.]
Running reward for the 4 bandits: [ -1. 277.   2.   7.]
Running reward for the 4 bandits: [ -1. 325.   0.   7.]
Running reward for the 4 bandits: [  0. 369.   3.   9.]


Running reward for the 4 bandits: [  0. 418.   3.  10.]
Running reward for the 4 bandits: [ -2. 461.   1.  13.]
Running reward for the 4 bandits: [ -2. 505.   3.  15.]
Running reward for the 4 bandits: [ -3. 553.   4.  15.]
Running reward for the 4 bandits: [ -3. 601.   5.  16.]


Running reward for the 4 bandits: [ -5. 648.   6.  16.]
Running reward for the 4 bandits: [ -3. 693.   7.  16.]
Running reward for the 4 bandits: [ -3. 740.   6.  18.]
Running reward for the 4 bandits: [ -3. 786.   6.  18.]
Running reward for the 4 bandits: [ -6. 832.   6.  19.]
Running reward for the 4 bandits: [ -6. 880.   6.  19.]


The agent thinks bandit 2 is the most promising....
...and it was right!
